# Collaborate and configure a distributed GIS

## Preliminary steps
Import the Python API library

In [ ]:
from arcgis.gis import *
import os
from zipfile import ZipFile
import json
download_path = '/Users/atma6951/Documents/temp/e_configs'

Connect with city GIS and Dept. GIS

In [ ]:
city_gis = GIS("https://dev005219.esri.com/portal",admin_username, admin_password)
dept_gis = GIS(city_url, admin_username, admin_password)

## Start collaboration

### Create a collaboration with city GIS as host

#### collab parameters

In [ ]:
city_collab_group = city_gis.groups.search('title:City {} dept. collaboration'.
                                               format(city_name.replace("_"," ")))
collab_name = 'LA ' + city_name.replace("_", " ") + " collab"
collab_description = "Distributed GIS and data sharing initiative"+
                        "between LA city and its constituent departments"
collab_wksp = 'LA ' + city_name.replace("_", " ") + " wksp"
collab_wksp_desc = 'Workspace to share data'

####

In [ ]:
city_collab = city_gis.collaborations.create(collab_name, collab_description, collab_wksp,
                                         collab_wksp_desc, county_collab_group[0].id,
                                         "Administrator", "Administrator", "admin@dmail.com")

### Invite dept. GIS to the collaboration

In [ ]:
city_collab_wksp = city_collab.workspaces[0]
config_json = [{city_collab_wksp['id']:"sendAndReceive"}]

city_collab_invite = city_collab.invite_participant(dept_gis._url, config_json, 24)

### Create a group for collaboration in dept. GIS

#### intermediate - create group

In [ ]:
title = 'LA city GIS collaboration'
tags = 'data, sharing, open data, sync'
description = 'Group enabling GIS collaboration with the city of Los Angeles'
snippet = 'This group contains content shared by and with the GIS of city of Los Angeles'

collab_group = dept_gis.groups.create(title, tags, description, snippet, access='public',
                                      thumbnail='collab_group.png')

### Accept invitation in dept. GIS

In [ ]:
dept_accept_response = dept_gis.collaborations.accept_invitation('Administrator', 
                                                                 'Administrator', 
                                                                 "admin@domain.com",
                                              invitation_file=city_collab_invite)

#generate response file
dept_collab = dept_gis.collaborations.collaborations[0]
invite_response_file = dept_collab.export_invitation(os.getcwd())

#add group to collaboration workspace
dept_collab_wksp = dept_collab.workspaces[0]
group_add_result = dept_collab.add_group_to_workspace(collab_group, dept_collab_wksp)

### Accept response back in City GIS

In [ ]:
city_accept_status = city_collab.import_invitation_response(invite_response_file)

## Style and configure the dept. GIS

### Get config file

In [ ]:
gsr = dept_gis.groups.search("title: LA city GIS collaboration", max_groups=1)
config_item = [i for i in dept_collab_group.content() if i.title.endswith("config")][0]

#download config file
config_file = config_item.download(download_path)

#extract config
zf = ZipFile(config_file)
zf.extractall(download_path)

### Set logo, banner, background

In [ ]:
dept_gis.ux.set_logo(download_path + '/logo.png')

dept_gis.ux.set_background(download_path + "/background.png")

dept_gis.ux.set_banner(download_path + "/banner.png")

### Set name, description, extent, featured content

In [ ]:
dept_gis.ux.set_name_description(city_name.replace("_", " ").title() + " dept GIS",
                                     "Distributed GIS between city govt. and departments")

dept_gis.update_properties({'defaultExtent': config_dict['extent']})

param_dict = {"homePageFeaturedContent":"id:{}".format(collab_group.groupid),
                  "homePageFeaturedContentCount":12}
dept_gis.update_properties(param_dict)

This is the end of configuration. The script proceeds to repeat this for the next department.